In [1]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

2.0.1+cu118
0.15.2+cu118


In [2]:
import matplotlib.pyplot as plt
import torch
import torchvision
from torch import nn
from torchvision import transforms
from torchinfo import summary

In [6]:
try:
    from going_modular import data_setup, engine
except:
    print("downloading...")
    print()
    !git clone https: // github.com/mrdbourke/pytorch-deep-learning
    !mv pytorch-deep-learning/going_modular.
    !rm -rf pytorch-deep-learning
    from going_modular import data_setup, engine

    print("done")

In [7]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [8]:
def set_seeds(seed: int = 42):
    """sets random sets for torch operation
    
    Args:
         seed (int, optional) : Random seed to set. default 42."""
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)


In [9]:
import os
import zipfile
from pathlib import Path
import requests


# our source: https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip

def download_data(source: str,
                  destination: str,
                  remove_source: bool = True) -> Path:
    """download a zipped file from the source and unzips it to the destination."""
    # setup path to data folder
    data_path = Path("data/")
    image_path = data_path / destination

    # if the image folder doesn't exist, create it
    if image_path.is_dir():
        print(f"[info] {image_path} directory already exit, skipping download.")
    else:
        print(f"[info] did not find {image_path} directory, creating one...")
        image_path.mkdir(parents=True, exist_ok=True)

        # download the target data
        target_file = Path(source).name
        with open(data_path / target_file, "wb") as f:
            request = requests.get(source)
            print(f"[info] downloading {target_file} from {source} ...")
            f.write(request.content)

        # unzipping
        with zipfile.ZipFile(data_path / target_file, "r") as zip_ref:
            print(f"[info] Unzipping {target_file} data ...")
            zip_ref.extractall(image_path)

        # removing source
        if remove_source:
            os.remove(data_path / target_file)
    return image_path


image_path = download_data(
    source="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
    destination="pizza_steak_sushi",
    remove_source=True)
image_path

[info] data\pizza_steak_sushi directory already exit, skipping download.


WindowsPath('data/pizza_steak_sushi')

### create DataLoader with manual transform

In [10]:
## create DataLoader with manual transform
# setup directories
train_dir = image_path / "train"
test_dir = image_path / "test"

train_dir, test_dir

(WindowsPath('data/pizza_steak_sushi/train'),
 WindowsPath('data/pizza_steak_sushi/test'))

In [17]:
# setup imagenet normalization

from torchvision import transforms

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# create transforms pipeline manually
manual_transforms = transforms.Compose([transforms.Resize((224, 224)),
                                        transforms.ToTensor(),
                                        normalize])
print(manual_transforms)
from going_modular import data_setup

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=manual_transforms,
                                                                               batch_size=32,
                                                                               num_workers=0)
train_dataloader, test_dataloader, class_names

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=warn)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)


(<torch.utils.data.dataloader.DataLoader at 0x1fbd8ad2910>,
 ['pizza', 'steak', 'sushi'])

### automaticlly method 

In [19]:
train_dir = image_path/"train"
test_dir = image_path/"test"
import torchvision
weights = torchvision.models.EfficientNet_B0_Weights.DEFAULT # means best available
automatic_transforms = weights.transforms()
print(automatic_transforms)

train_dataloader, test_dataloader, class_names = data_setup.create_dataloaders(train_dir=train_dir,
                                                                               test_dir=test_dir,
                                                                               transform=automatic_transforms,
                                                                               batch_size=32,
                                                                               num_workers=0)
train_dataloader, test_dataloader, class_names

ImageClassification(
    crop_size=[224]
    resize_size=[256]
    mean=[0.485, 0.456, 0.406]
    std=[0.229, 0.224, 0.225]
    interpolation=InterpolationMode.BICUBIC
)


(<torch.utils.data.dataloader.DataLoader at 0x1fbd0602850>,
 ['pizza', 'steak', 'sushi'])